In [35]:
using CSV, DataFrames, Printf

In [36]:
@show primAuthor = "Ivan"
fileOwnPapers = "ISI-Teze/TZ.2024.11.29_6_IR.savedrecs.txt"
#root = dirname(@__FILE__)
root = dirname("/home/lali/TITAN-ROG-sync/julia/")
@show inputAIS = joinpath(root, "ISI/AI_factor_full_1997-2024_plus.txt")
@show inputISI = joinpath(root, "ISI/IF_factor_full_1997-2024_plus.txt")

@show inputMINE = joinpath(root, fileOwnPapers);

primAuthor = "Ivan" = "Ivan"
inputAIS = joinpath(root, "ISI/AI_factor_full_1997-2024_plus.txt") = "/home/lali/TITAN-ROG-sync/julia/ISI/AI_factor_full_1997-2024_plus.txt"
inputISI = joinpath(root, "ISI/IF_factor_full_1997-2024_plus.txt") = "/home/lali/TITAN-ROG-sync/julia/ISI/IF_factor_full_1997-2024_plus.txt"
inputMINE = joinpath(root, fileOwnPapers) = "/home/lali/TITAN-ROG-sync/julia/ISI-Teze/TZ.2024.11.29_6_IR.savedrecs.txt"


In [37]:
@show AIFirstYear = 1997;
@show AILastYear = 2024;

AIFirstYear = 1997 = 1997
AILastYear = 2024 = 2024


In [38]:
AIS = CSV.read(inputAIS, DataFrame; header=true, delim='\t');

In [39]:
ISI = CSV.read(inputISI, DataFrame; header=true, delim='\t');

In [40]:
#@show AIS[8157, [1, 3, 5, 6,    29, 30]];
#@show ISI[8157, [1, 3, 5, 6,    29, 30]];
AIS = dropmissing(AIS, :Journal)
ISI = dropmissing(ISI, :Journal);

In [41]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true, delim='\t');
MINE = MINEraw[:, [2, 9, 25, 33, 44, 47, 48, 54, 55, 56, 57]];
#                 AU TI  RP  NR  J9  PY  VL  BP  EP  AR  DI

sort!(MINE, [:PY], rev = true)
eltype.(eachcol(MINE));

MINEraw=nothing;

In [42]:
@show MINE[1, [:AU, :J9, :PY]];

MINE[1, [:AU, :J9, :PY]] = DataFrameRow
 Row │ AU                                 J9                 PY
     │ String                             String31           Int64
─────┼─────────────────────────────────────────────────────────────
   1 │ Lebière, PG; Ivan, R; del Pino, …  COLLOID SURFACE A   2023


In [43]:
#   PT	AU	BA	BE	GP	AF	BF	CA	TI	SO	SE	BS	LA	DT	CT	CY	CL	SP	HO	DE	ID	AB	C1	
#   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  

#   C3	RP	EM	RI	OI	FU	FP	FX	CR	NR	TC	Z9	U1	U2	PU	PI	PA	SN	EI	BN	J9	JI	PD	
#   24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  

#   PY	VL	IS	PN	SU	SI	MA	BP	EP	AR	DI	DL	D2	EA	PG	WC	WE	SC	GA	PM	OA	HC	HP	DA	UT
#   47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71

#   ######  2   9   32  44  46  47  48  54  55  56  57
#   ######  AU  TI  CR  J9  PD  PY  VL  BP  EP  AR  DI

###########################################################  
####################  Finished loading   ########################## 
###########################################################

In [44]:
rename!(MINE,:"J9" => :"Journal")

insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);


Găsim numarul de autori și prim autor

In [45]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    #println( titlecase(MINE[row_index, :AU]))
    #println(findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1])
    
    if findfirst(primAuthor, titlecase(MINE[row_index, :AU]))[1] == 1
        MINE[row_index, :PRIM] = 1
    elseif !ismissing(lucrare[:RP])
        if occursin(primAuthor, lucrare[:RP])
            MINE[row_index, :PRIM] = 1
        end
    end

    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

În caz că lipsește un publication year:

In [46]:
#MINE.PY = Int64.(MINE.PY)
#MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

Subseturi (?):

In [47]:
#AISsubset = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#size(AISsubset, 1);

In [48]:
#ISIsubset = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), ISI)
#size(ISIsubset, 1);

Căutare MINE

In [49]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
    #for AISrow in eachrow(AISsubset)
        if( lucrare[:Journal] == AISrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
        end
    end
    for ISIrow in eachrow(ISI)
    #for ISIrow in eachrow(ISIsubset)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            if (lucrare[:PY] >= AIFirstYear)
                anString = string(lucrare[:PY])
            else
                anString = string(AIFirstYear)
            end
            if (lucrare[:PY] > AILastYear)
                anString = string(AILastYear)
            end
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
        end
    end
end

Rezultate MINE:

In [50]:
I = sum(MINE[!, :AINEFF]);
P = sum(MINE[!, :PRIM] .* MINE[!, :AI]);

sumAI = sum(MINE[!, :AI]);

I, P, sumAI

(0.8799018648018648, 3.022, 4.927)

In [51]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    @printf("%10s --- %20s --- %4d --- IF=%6.3f --- AIS=%6.3f\n",MINE[row_index, :AU][1:10], MINE[row_index, :Journal], MINE[row_index, :PY], MINE[row_index, :IF], MINE[row_index, :AI]);
end
sumAI

 Lebière,  ---    COLLOID SURFACE A --- 2023 --- IF= 4.900 --- AIS= 0.650
Ivan, R; P ---           SCI REP-UK --- 2023 --- IF= 3.800 --- AIS= 1.059
Ivan, R; P ---        APPL SURF SCI --- 2020 --- IF= 6.707 --- AIS= 0.878
 Queraltó, ---             CRYSTALS --- 2020 --- IF= 2.589 --- AIS= 0.482
Ivan, R; d --- J PHOTOCH PHOTOBIO A --- 2020 --- IF= 4.291 --- AIS= 0.520
Ivan, R; P ---          J MATER SCI --- 2019 --- IF= 3.553 --- AIS= 0.565
del Pino,  ---        APPL SURF SCI --- 2019 --- IF= 6.182 --- AIS= 0.773


4.927